# Lab 3 - Logistic Regression & Stochastic Gradient Descent

**Due: October 24th at 11:59pm**

***
## Learning Goals

The goals of this lab is to  implement the Logistic Regression classification model and the stochastic gradient descent algorithm. 

As with any programming assignment, you'll also be practicing and improving your general CS skills, like problem decomposition, algorithmic thinking, implementation and testing, language syntax, etc..  Here are some of the specific things you should learn while completing this assignment:


* How to implement the Logistic Regression classification model
* How to implement the Stochastic Gradient Descent optimization algorithm
* How to implement your own class to match the interface used by SKLearn
* More practice with custom data files, Pandas, Numpy, SKLearn, etc.

As usual, look for the TODO markers in the notebook below. Please be sure to *remove* the TODO markers as you complete each aspect (i.e. don't leave a TODO that's for something you've actually done, that's poor style because it's confusing to anyone reading your code/documentation/etc.)

This lab is intended to be done with a partner (i.e. teams of two); partners will be assigned based on the survey.  You'll have approximately **two weeks** to complete this lab, so be sure to start early and plan properly to get it all done in a timely fashion.  **NOTE: this lab is due after you get back from Fall Break, but I strongly encourage you to finish it and turn it in before you leave!**

As with any longer term assignment, it's a chance to train your time management skills.  It is recommended that you complete the implementation of the `LogisticRegression` class (everything in this file up to **Further testing**) during the first week of the lab.  You should be able to run and test the entire `toy` data set.  Week 2 of the lab can then focus on debugging, applying your model to more difficult data sets, adding an extension to your implementation.


***
## Implementation Overview

Unlike the previous lab, we are largely leaving the design of your solution up to you.  The only requirement is that you should have an interface that mimics SciKit Learn models.

As a reminder, however, we **strongly encourage** you to follow good development practices by doing a top-down design, creating stubs, testing as you go, etc.

You will design a class called `LogisticRegression` that will maintain necessary data and add methods to train the model and classify data points.  This has been stubbed out with minimal code below; you will need to add both data members and methods.


In [1]:
# CPSC 66 - Machine Learning
# Lab 3
# Scaffolding by Prof. Ameet Soni & Ben Mitchell
# Assignment completed by: Nina Zhuo
# Resources used: 
#   <List any resources you used beyond the ones posted on Blackboard>
#   <This can include books, websites, other students, etc.>

## Getting Started 

You will write your program implementation in this file.  In addition, you will utilize the following input files located in the `data` directory:
 
 * `simple_data.csv` - a toy data set to help debug your algorithm
 * `mammal_train.csv` and `mammal_test.csv` - pre-split data sets for zoo animals that are either mammals or not
 * `titanic.csv` - data set of outcomes for individuals on the Titanic 

Let's import libraries up top.  Feel free to add libraries.  You should not be using any machine learning libraries in your actual Logistic Regression class implementation.  We have included SciKit Learn's `SGDClassifier` below as a debugging tool only.  If implemented correctly, your solution should produce the same weights and predictions as `SGDClassifier`. 

In [2]:
import math
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

from warnings import filterwarnings
filterwarnings('ignore') #Removes warnings about convergence

For a stopping criteria, you will use 100 iterations instead of calculating weight changes.  Rather than hardcode, we'll set a global variable. **You will probably actually want to change this during debugging.** For example, you can set it to 1 and compare your weights to the correct implementation (`SGDClassifier`) after just one iteration of stochastic gradient descent.

In [3]:
MAXITER = 100

## SGDClassifier

As mentioned, we'll use the `SGDClassifier` from SciKit Learn as a baseline comparison tool.  This box imports some data and creates a classifier.  Refer to this when you write more tests later.

In [4]:
toyX = pd.read_csv('data/simple_data.csv') # open the csv file to see the data
toyY = toyX.pop("stroke") #remove the label column

alpha = .1 #learning rate, you can change this

"""
 Create a SGDClassifier with the following settings: use logistic regression for MAX_ITER 
 iterations.  Does not shuffle the data (so we can directly compare results to our algorithm)
 and use learning rate, alpha, that will not change.  For now, do not impose a penalty on weights. 
"""
sgd = SGDClassifier(loss='log', max_iter=MAXITER,shuffle=False, tol=None, \
                    penalty='none', learning_rate='constant', eta0 = alpha)
sgd.fit(toyX,toyY) # train the model

SGDClassifier(eta0=0.1, learning_rate='constant', loss='log', max_iter=100,
              penalty='none', shuffle=False, tol=None)

## Provided Helper Functions

These are helper functions provided for you, but feel free to add to or modify them if you like

In [19]:
def printWeights(features, weights):
    """
    Pretty-print the model weights.
    features contains the name of each feature and weights is an array type of the same length.
    You may need to modify this function depending on your implementation
    """
    if "bias" not in features:
        print("Assuming the last weight is bias term; modify the printWeights function if this is not true")
        features = list(features)+["bias"]
    if len(weights) != len(features):
        print("ERROR: printWeights() called with non-matching feature and weight vector lengths")
        return
    print("\t%30s %10s" % ("Feature", "Weight"))
    for i in range(len(features)):
        print("\t%30s %10.3f" % (features[i], weights[i]))

def normalize(X):
    """
    You will get overflow problems when calculating exponentials if 
    your feature values are too large.  This function adjusts all values to be
    in the range of 0 to 1 for each column.
    """
    
 #  assert (X.min() != X.max()) # make sure the range is non-0 (otherwise, we'll end up with NaNs)
        
    X = X - X.min() # shift range to start at 0
    normalizedX = X/X.max() # divide by possible range of values so max is now 1
    return normalizedX

### Examining the output of SGDClassify

The cell below shows how we can get the parameter values out of the SKLearn model once it's been trained (_note that the training happened several cells prior to this one_).  It also shows the accuracy of the model, which works just like it does with other SKLearn models.

Note that we evaluate accuracy on the training data here, since we never split this set; this is fine for the purposes of testing the correctness of our training algorithm, but be careful not to do this when testing on real data.

In [6]:
accuracy = sgd.score(toyX, toyY) # note that we're testing on the training data here...
print("Accuracy from SKlearn SGDClassifier on toy data: %.2f\n" % (accuracy))

print("Coefficients and intercept for SKlearn SGDClassifier: ")
weights = list(sgd.coef_[0]) + list(sgd.intercept_) #sklearn stores these weights separately, so lets combine them into one list
printWeights(toyX.columns,  weights)

Accuracy from SKlearn SGDClassifier on toy data: 1.00

Coefficients and intercept for SKlearn SGDClassifier: 
Assuming the last weight is bias term; modify the printWeights function if this is not true
	                       Feature     Weight
	                           age      1.967
	                    cholestrol     -1.888
	                          bias     -4.422


***
## LogisticRegression class

Define the class and methods here.  Practice incremental development.  If you prefer to code in a native Python environment instead of Jupyter, you can convert this to a Python file (see instructions [here](https://jupyterlab.readthedocs.io/en/stable/user/export.html)).  If you do so, replace the code below with an import statement.  For example, if you name the class definition file `LogRegressionFile.py`, then import everything using 

```from LogRegFile import *```

You will still need to run the tests and provide analysis in this file.

Begin by defining the constructor.  Add any data members you want to keep track of.  At a minimum, you should store the weights as well as the parameters given below (learning rate, the names of all features, and the class labels).  You can assume a binary classification task.  Remember to rerun this code block if you change any of the data members. 


Also, remember that because Python is a dynamically typed language, **an object's type is that of the class _when the object was instanciated._**  In _particular_, if you construct and object and then subsequently add things to the class definition (which is actually overriding the original class definition with a new one that extends the old one but uses the same name), your object will not have those newly added things.  You need to create a new object after executing the new class definition before things will work.

The example code below separates the class definition into multiple cells to encourage modular development, but this means that you'll need to move/copy/re-run the testing example given below the initial class definition if you want the object `lr` to actually contain your later additions (we've actually done the "copy" option for you in the scaffolding, but it's still a potential issue to be aware of as you modify and run things).

Note: in the reference implementation, we assume the first item in the list `label` is the negative class and the second value is the positive.  This is arbitrary. If you flip this, you will get the same accuracy but potentially weights with the opposite magnitude (i.e., all the weights will be negated).  You can assume that all features are numeric for now.  You can also ignore `lmbda` - this is for a possible extension suggested at the bottom of the write up.

In [7]:
class LogisticRegression(object):

    def __init__(self, features, labels, alpha, lmbda = 0):
        """
        Constructor.  Create class variables here.  At a minimum, you'll need
        to store the learning rate (alpha) and weights.  You may also choose to add meta data
        e.g., feature names, labels, etc. depending on your needs below
        """
        self.alpha = alpha # learning rate        
        self.features = np.array(features) # names of all features
        self.labels = np.array(labels) # class labels
        self.lmbda = lmbda
        self.w = [] # weight vector

In [8]:
lr = LogisticRegression(toyX.columns, toyY.unique(), alpha) # create a model similar to the SGDClassifier above

print(lr)
print("features = ", lr.features)
print("labels = ", lr.labels)
print("alpha = ", lr.alpha)
print("lmbda = ", lr.lmbda)
print("w = ", lr.w)

features =  ['age' 'cholestrol']
labels =  ['Neg' 'Pos']
alpha =  0.1
lmbda =  0
w =  []


***
### Training algorithm

Your class should have a method `fit(X,y)` that trains the model weights.  It takes in training examples `X` and their labels `Y` which can be any array-like object you choose (Pandas data frames, numpy arrays, Python lists, etc).  It does not return anything.  It will use **stochastic gradient descent** for its implementation.  Recall the math for gradient descent from class, as well as the fact that the S in **S**GD means we will update weights after each example seen.  As a reminder of the equations we covered:

**Stochastic Gradient Descent**

for each weight $w_j$ in the weight vector, use **one** training example with index $i$ (then repeat with a different $i$):

$$ w_j \leftarrow w_j - \alpha \left( P(y=1 | \vec{x}^i) - y^i \right) \vec{x}_j^i $$

where

$$P(y=1 | \vec{x}^i) = \frac{1}{1 + e^{- \left( \vec{w} \cdot \vec{x}^i \right) }} $$


Rember that $\vec{u}\cdot\vec{v}$ is the same as $\vec{u}^\top\vec{v}$; it's just two different notations for the same operation (this is important to know as different references may use different notation).

***
Here is some pseudocode for the algorithm:

```
initialize weights to 0
while not converged:
    initialize gradient to 0 for each feature (g = [0,...,0])
    for each training example xi:
        p = prob(w, xi) //predict probabilty of positive under current model
        error = p-yi // calculate error in prediction
        for each feature j:
            g[j] = error*xi[j]
            w[j] -= alpha*g[j] //update weights
```

***
For simplicity, use a maximum number of iterations of the outer loop as the convergence criteria (the number should be set by the constant `MAXITER`, defined above).

Note that there must be a weight for the bias/intercept term.  To accommodate, you'll need to either add a fake value of 1 to your training set examples to create an extra feature, or explicitely add a bias term (and associated weight) to your class data members.  The extra 1 to make the vector lengths match _is not_ built-in to your training set. 

Make sure not to modify the original input data when you do this(i.e., you may need to create a copy of `X`).  For example, if you decide to make `X` a Pandas dataframe, you can create a bias "feature" like so:

```
copyX = X.copy() // makes a "deep" copy of a pandas dataframe
copyX["bias"] = 1 // adds a column "bias" and sets each row's value for that new colum to 1 (modifies copyX but not X)
```

You will probably find the [Numpy `dot`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) function useful as well as the [`math.exp()`](https://www.w3schools.com/python/ref_math_exp.asp) function.

In [9]:
# This is effectively adding an extra method to the existing class
# (though technically it's actually a new class inheriting from the previous one,
# but with the same name so it hides the old one)
class LogisticRegression(LogisticRegression):
    def fit(self, X, Y):
        """
        train the model weights.  Initialize the weights to 0 first and use gradient ascent 
        using the training examples {X,Y}
        
        X = training examples
        Y = array-like object with labels
        """        
        
        # create deep copy of X and add bias column
        copyX = X.copy()
        copyX["bias"] = 1
        
        # create copy of Y to index into
        Y = Y.tolist()
                
        # initialize weights to zero
        self.w = [0]*(len(self.features) + 1)
        
        # set convergence criteria
        iters = 0
        while iters < MAXITER:
            
            # initialize gradient to 0 for each feature
            g = [0]*len(self.w)

            for i in range(len(X)):
                
                # set xi
                xi = copyX.iloc[i].to_numpy()
                
                # calculate probability and error for each example
                p = self.prob(xi)
                                
                if Y[i] == self.labels[0]:
                    error = p
                else:
                    error = p - 1

                # updates weights
                for j in range(len(self.w)):
                    g[j] = error*xi[j]
                    self.w[j] -= self.alpha*g[j]
            
            iters += 1
                                    
    def prob(self, xi):
        if (len(self.w)) > len(xi):
            xi = np.append(xi, 1)    
    
        p = -1*np.dot(self.w, xi)
        p = np.exp(p)
        p = 1/(1+p)
            
        return p

#### Tests for training

Let's compare the weights of your classifier to the SciKit Learn implementation.  Your implementation should match the `SGDClassifier` assuming you go for the same number of iterations.  If you have the opposite magnitude that is okay. Be sure and try just a few iterations at first to see where you are deviating; you can calculate the values by hand to make sure your math is correct.  Another suggestion is create an even smaller data set with e.g., 2 data points.

In [10]:
#TODO: add further testing code here to help debug your training algorithm
lr = LogisticRegression(toyX.columns, toyY.unique(),alpha) # create a model similar to the SGDClassifier above
lr.fit(toyX,toyY) #fit the same data as the SGDClassifier

print("Coefficients and intercept for SKlearn SGDClassifier: ")
weights = list(sgd.coef_[0]) + list(sgd.intercept_) #sklearn stores these weights separately, so lets combine them into one list
printWeights(toyX.columns,  weights)

print()
print("Coefficients and intercept for Logistic Regression: ")
printWeights(toyX.columns,  lr.w)

Coefficients and intercept for SKlearn SGDClassifier: 
Assuming the last weight is bias term; modify the printWeights function if this is not true
	                       Feature     Weight
	                           age      1.967
	                    cholestrol     -1.888
	                          bias     -4.422

Coefficients and intercept for Logistic Regression: 
Assuming the last weight is bias term; modify the printWeights function if this is not true
	                       Feature     Weight
	                           age      1.967
	                    cholestrol     -1.888
	                          bias     -4.422


***
### Prediction/Inference 

You will implement two different functions to provide a prediction for a novel example: `predict_prob` returns the probability of the positive label, while `predict` will return a discrete label.

***
**Predicting a probability**
To predict the probability of a positive label, use the equation from class: 

$$P(y=1|\vec{x}) = \frac{1}{1+e^{-\left(\vec{w}\cdot\vec{x}\right)}}$$

_Note that this is closely related to a part of the SGD algorithm from above; consider ways to make your implementation modular to avoid code replication._

***
**Predicting a label**
To produce a discrete prediction, you can either compare the positive vs negative probabilities and pick the larger (HINT: they sum to 1, so you just need to know if the probability of a positive is greater than 0.5).  Another common technique is just to see if $w^Tx$ is greater than 0: a value of greater than 0 yields a probability score greater than 0.5, and a negative value gives a probability of less than 0.5.  

_Note again that you can and should use your `predict_prob` function here to avoid code replication!_


In [11]:
class LogisticRegression(LogisticRegression):

    def predict_prob(self, X):
        """
        Returns a 2D array of size [num_examples, num_classes] containing the probability 
        of each prediction.  Since logistic regression is a binary classifier, there should only be
        two columns.  That is, give the probability that the model would assign the two possible labels.
        """
        
        # set up 2D array with n subarrays with length m 
        # n = num_examples, m = num_classes
        n, m = (len(X), len(self.labels)) 
        arr = [[0 for i in range(m)] for j in range(n)] 
        
        
                    
        for i in range(n):
            # get row
            xi = X.iloc[i].tolist()
            
            # calculate probability
            p = self.prob(xi)
            
            # save probability
            arr[i][1] = p
            arr[i][0] = 1 - p
                    
        return arr
    
    def predict(self, X):
        """
        classify the examples in X.  Returns the predicted label for each example in X (be sure to use the stored
        labels array from the constructor instead of just returning 0s and 1s).  The length of 
        the returned array should be the same as the number of examples in X.
        """
        
        # get 2d array of probability of each prediction
        prob = self.predict_prob(X)
        
        # set threshold for binary classification
        div = 0.5   
        
        # initialize array storing predicted labels
        arr = [0]*len(X)
        
        # determines predicted label for each example
        for i in range(len(X)):
            if prob[i][1] > div:
                arr[i] = self.labels[1]
            else:
                arr[i] = self.labels[0]
                        
        return arr


#### Tests for prediction
Now let's test the implementation on the toy data set to see if matches.  Since are making predictions on the training set and the problem is fully linear, you should get 100% accuracy. 

In [12]:
### Tests for SKLearn
predictions = sgd.predict(toyX)
score = sgd.score(toyX,toyY)
print("Accuracy and Predictions for **SKlearn SGDClassifier**: ")
print("Accuracy: %0.2f%%" % (score*100))

print("%12s %12s" % ("Prediction", "Truth"))
print("-"*25)
for i in range(len(predictions)):
    print("%12s %12s" % (predictions[i], toyY[i]))
print()

### Tests for your implementation for comparison
lr = LogisticRegression(toyX.columns, toyY.unique(),alpha) # create a model similar to the SGDClassifier above
lr.fit(toyX,toyY) #fit the same data as the SGDClassifier
predictions = lr.predict(toyX)

if predictions == None:
    print("Error: LogisticRegression.predict() not yet implemented")
else: 
    score = len(toyY[toyY == lr.predict(toyX)])/len(toyY)
    print("Accuracy and Predictions for **Custom Logistic Regression**: ")
    print("Accuracy: %0.2f%%" % (score*100))

    print("%12s %12s" % ("Prediction", "Truth"))
    print("-"*25)
    for i in range(len(predictions)):
        print("%12s %12s" % (predictions[i], toyY[i]))

Accuracy and Predictions for **SKlearn SGDClassifier**: 
Accuracy: 100.00%
  Prediction        Truth
-------------------------
         Neg          Neg
         Pos          Pos
         Neg          Neg
         Pos          Pos
         Neg          Neg
         Pos          Pos
         Neg          Neg

Accuracy and Predictions for **Custom Logistic Regression**: 
Accuracy: 100.00%
  Prediction        Truth
-------------------------
         Neg          Neg
         Pos          Pos
         Neg          Neg
         Pos          Pos
         Neg          Neg
         Pos          Pos
         Neg          Neg


***
## Further testing: Mammal Prediction Task

Let's try some real data.  Test your implementation on the Mammal data set in `data` (`mammal_train.csv` and `mammal_test.csv`).  This is a simplified version of the [zoo animal](https://archive.ics.uci.edu/ml/datasets/Zoo) data set where all animals in category 1 have been given the label `Mammal` and all other categories are `Other`.  The data has already been divided into training and testing sets for you.  We've provided you code to load the data set and normalize the features (make sure they all range from 0 to 1).  You should write code that at a minimum:

* trains your classifier
* prints out the weights 
* checks the accuracy on the test set

We also recommend comparing your weights and your predictions with the `SGDClassifier` and seeing if the results weights make sense in the real world (i.e., are these features correlated with mammals in real life?).

In [13]:
# load the data sets, train your model and print out the weights and predictions.  
# Play around with alpha; you should be able to get 100% Accuracy

mammal_train = pd.read_csv('data/mammal_train.csv')
mammal_test = pd.read_csv('data/mammal_test.csv')

# Some minimal data processing.  Let's shuffle the examples so they don't appear in order
# Then, we should normalize the legs feature so all values are between 0 and 1
mammal_train = mammal_train.sample(frac=1).reset_index(drop=True)
mammal_test = mammal_test.sample(frac=1).reset_index(drop=True)
mammal_train["legs"] /= 8 #all features are normalized to be between 0 and 1 except for legs, so let's normalize it 
mammal_test["legs"] /= 8 #all features are normalized to be between 0 and 1 except for legs, so let's normalize it 

# separate out labels from training data
ytrain = mammal_train.pop("animalType")
ytest = mammal_test.pop("animalType")

# set up sgd model
sgd.fit(mammal_train, ytrain) # train the model
print("Coefficients and intercept for SKlearn SGDClassifier: ")
weights = list(sgd.coef_[0]) + list(sgd.intercept_) #sklearn stores these weights separately, so lets combine them into one list
printWeights(mammal_train.columns,  weights)
print()

# compare to implemented lr model
lr = LogisticRegression(mammal_train.columns, ytrain.unique(),alpha)
lr.fit(mammal_train, ytrain) 
print()
print("Coefficients and intercept for Logistic Regression: ")
printWeights(mammal_train.columns,  lr.w)
print()

# run lr model on training set
predictions = lr.predict(mammal_test)
if predictions == None:
    print("Error: LogisticRegression.predict() not yet implemented")
else: 
    score = len(ytest[ytest == lr.predict(mammal_test)])/len(ytest)
    print("\nAccuracy and Predictions on Testing Set: ")
    print("Accuracy: %0.2f%%" % (score*100))

    print("%12s %12s" % ("Prediction", "Truth"))
    print("-"*25)
    for i in range(len(predictions)):
        print("%12s %12s" % (predictions[i], ytest[i]))

Coefficients and intercept for SKlearn SGDClassifier: 
Assuming the last weight is bias term; modify the printWeights function if this is not true
	                       Feature     Weight
	                          hair     -2.930
	                      feathers      1.311
	                          eggs      3.691
	                          milk     -4.335
	                      airborne      1.432
	                       aquatic      0.629
	                      predator      0.616
	                       toothed     -0.377
	                      backbone     -0.442
	                      breathes     -0.484
	                      venomous      1.537
	                          fins      0.580
	                          legs      0.507
	                          tail      0.199
	                      domestic      0.174
	                       catsize     -1.975
	                          bias      1.803


Coefficients and intercept for Logistic Regression: 
Assuming the last weight

***
## Further testing: Titanic Data Set

Now, we'll try a harder task: the titanic data set in `data/titanic.csv`.

The dataset in question is based on the passenger list from the Titanic; each passenger has a record, with some known information, along with whether or not that passenger survived the disaster when the ship sank.  The standard supervised learning task for this data set is to predict whether a given passenger survived given the rest of their information.   If who survived was random, this shouldn't work (we wouldn't be able to do better than chance), but if it's not random then we might be able to build a successful classifier, and ideally even understand what kind of traits the survivors had in common.

This data was first posted on Kaggle: https://www.kaggle.com/c/titanic/data, and that site also has a description of what the columns mean.  Note however t?hat the version of the data we've provided you has been cleaned and pre-processed to remove missing values, non-numeric features, etc.

Below write some test code and see if you can use your implementation to test the robustness of your implementation.  Be sure to do a proper train/test evaluation. Does changing the learning rate impact performance?  Do the feature weights make sense for this task?  Do we get interesting results if we examine the types of errors?   **Write 2-3 sentences describing any interesting findings for this data set** in the "Analysis" area below the code cell.

In [14]:
def getTestAndTrain(data, trainFrac =0.7, seed =0):
    # let's shuffle this and split it into train and test sets:
    shuffled = data.sample(frac=1, random_state=seed) # randomly re-order the examples
        # note: the 'frac=1' means use all the examples; also note this creates a new "view", 
        # it doesn't do a deep copy, so the row index values in `shuffled` will be out of order
    trainCount = int(len(data) * trainFrac) # figure out how many examples we want in each set
    train = shuffled[:trainCount] # slice out the examples we'll use for training (again, creates a view)
    test = shuffled[trainCount:] # use the remainder for testing

    return [train, test]

In [21]:
# divide data into test and training sets
titanicData = pd.read_csv('data/titanic.csv')
sets = getTestAndTrain(titanicData)

# set up X and Y for testing and training sets
trainX = sets[0]
normalize(trainX)
trainY = trainX["Survived"]
trainX.drop(columns=["Survived"])

testX = sets[1]
normalize(testX)
testY = testX["Survived"]
testX.drop(columns=["Survived"])

# set up sgd model
print("Coefficients and intercept for SKlearn SGDClassifier: ")
sgd.fit(trainX, trainY) # train the model
weights = list(sgd.coef_[0]) + list(sgd.intercept_) #sklearn stores these weights separately, so lets combine them into one list
printWeights(trainX.columns,  weights)
print()

# compare to implemented lr model
print("Coefficients and intercept for Logistic Regression: ")
lr = LogisticRegression(trainX.columns, trainY.unique(), alpha)
lr.fit(trainX, trainY)
printWeights(trainX.columns, lr.w)
print()

# run lr model on training set
predictions = lr.predict(testX)
if predictions == None:
    print("Error: LogisticRegression.predict() not yet implemented")
else: 
    score = len(testY[testY == lr.predict(testX)])/len(testY)
    print("\nAccuracy and Predictions on Testing Set: %0.2f%%" % (score*100))

Coefficients and intercept for SKlearn SGDClassifier: 
Assuming the last weight is bias term; modify the printWeights function if this is not true
	                       Feature     Weight
	                      Survived    158.316
	                        Pclass    -26.654
	                           Sex     49.212
	                         SibSp    -10.660
	                         Parch     -2.991
	                          Fare     -0.029
	                          bias    -19.944

Coefficients and intercept for Logistic Regression: 
Assuming the last weight is bias term; modify the printWeights function if this is not true
	                       Feature     Weight
	                      Survived    166.658
	                        Pclass    -27.186
	                           Sex     54.009
	                         SibSp    -12.501
	                         Parch     -4.460
	                          Fare      0.030
	                          bias    -19.468


Accuracy and Pred

### Analysis

Unsurprisingly, increasing the value value of alpha decreases the overall accuracy of the model because the weights are taking larger steps away from the error, which gets us to the model faster, but sacrifices greater accuracy. Looking at the model, it makes sense that the sex of the passenger is weighted the highest in almost every iteration because when the life boats came out, women and children were prioritized over male passengers. When increasing the learning rate however, I was surprised to see that a higher fare contributed to greater likelihood of survival, when the opposite is true for a lower learning rate.

***
## Extensions

**You are required to pick and implement one of these extensions**.  It is up to you which one you choose - they are varying degrees of difficulty involved.  Feel free to implement more than one!

### Multiclass prediction

 Logistic Regression is designed for binary classification tasks.  However, we can design a fairly simple extension for multiclass prediction (as most machine learning packages do) known as 1-one-vs all prediction.  Review [this page](http://mlwiki.org/index.php/One-vs-All_Classification) for a description.  You can use the following pseudocode to structure your design:
``` 
    given training examples X,y
    for each class k
       create y_k where all instances of class k are positive and all other instances are negative
       train a logistic regression model on X,y_k --> model_k
    
    given a test example x_test
    run x_test on each model_k to give prob_k (probability of positive under model_k)
    return the k with the highest prob_k as the prediction
```

Pick a data set (we recommend the digits task from Lab 2B or the zoo data set provided in `data`).   The major deliverable is the code to implement this extension; no analysis is necessary except to explain what data set you used. 

### Penalize weights

Add an L2 penalty term to the training algorithm.  This requires minimal modification to your code.  We have already provided the option of specifying the $\lambda$ as part of the constructor for `LogisticRegression`.  Modify your `LogisticRegression` code above to use that `lmbda` and change your stochastic gradient descent algorithm to incorporate this into weight updates.  

Since this is a bit easier to implement, you'll need to **provide a few sentences of analysis**.  In particular, we want you to try out a few different $\lambda$ values and describe how it impacts performance as well as the weights (do they change significantly in one way or another?).  Does this align with our discussion on regularization in lecture?  Pick a data set for your analysis (probably titanic or some other difficult problem).  

### Multinomial Features and Data Cleanup

Download and clean up your own data set that uses multinomial features (discrete with more than 2 values).  Note: this option may be a bit trickier and open-ended than the other options, but is good practice for the final project.  

Test out the on your classifier implementation.  Your could should include non-trivial preprocessing and testing out different hyperparameters (e.g,. $\alpha$, `MAXITERS`).  At a minimum, we want to see how you adapt your model to work with "messy" data sets as a preparation for the final project.  You may way want to think about the Tennis data set (`data/tennis.csv`) as a thought experiment.  How can you use this data set in logistic regression?  In your analysis, include any clean up steps you performed and why.



In [22]:
def getModels(X, Y, classes):

        # stores all linear regression models for each class k
        models = [None]*len(classes)

        for k in range(len(classes)):
            # creates y_k where all instances of class k are > 0 and all other instances are < 0
            y_k = np.zeros(len(Y))
            for i in range(len(Y)):
                if Y.iloc[i] == classes[k]:
                    y_k[i] = 1
                    
            # train logistic regression model to find instances that are or not k
            model_k = LogisticRegression(X.columns, np.unique(y_k), alpha)            
            model_k.fit(X, y_k)
            models[k] = model_k
            
        return models

def predictLabel(classes, models, query):
            
        maxProb = 0
        label = 0
        
        # run query on each model_k
        for k in range(len(models)):
            model_k = models[k]
            probability = model_k.prob(query)            
            if probability > maxProb:
                maxProb = probability
                label = classes[k]
    
        return label

In [24]:
# divide data into test and training sets
zooData = pd.read_csv('data/zoo.csv')
sets = getTestAndTrain(zooData)
print

# set up X and Y for testing and training sets
trainX = sets[0]
normalize(trainX)
trainY = trainX["animalType"]
trainX.drop(columns=["animalType"])

testX = sets[1]
normalize(testX)
testY = testX["animalType"]
testX.drop(columns=["animalType"])

# get classes and train model
classes = trainY.unique()
models = getModels(trainX, trainY, classes)

# check accuracy
predict = []
for i in range(len(testX)):
    prediction = predictLabel(classes, models, testX.iloc[i])
    predict.append(prediction)
    
score = len(testY[testY == predict])/len(testY)
print("\nAccuracy and Predictions on Testing Set: %0.2f%%" % (score*100))


Accuracy and Predictions on Testing Set: 90.32%


### Extension Analysis

Clearly state what extension you did, what you learned, and a bit of analysis that is suggested as part of the descriptions above.  You shouldn't need more than 5 or 6 sentences of analysis (and probably less).

I did the Multiclass prediction extension on the zoo dataset in data/zoo.